In [1]:
import pandas as pd
import numpy as np

In [2]:
# %config InlineBackend.figure_format = 'svg'

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(6, 4)})

import functools

In [3]:
s_selected_columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV','DEC']

In [4]:
num_to_month = {
    1: 'JAN',
    2: 'FEB',
    3: 'MAR',
    4: 'APR',
    5: 'MAY',
    6: 'JUN',
    7: 'JUL',
    8: 'AUG',
    9: 'SEP',
    10: 'OCT',
    11: 'NOV',
    12: 'DEC'
}

ONI Update

In [5]:
def oni_mon_to_number(example) : 
    oni_mon_map = {
        'DJF' : 1,
        'JFM' : 2,
        'FMA' : 3,
        'MAM' : 4,
        'AMJ' : 5,
        'MJJ' : 6,
        'JJA' : 7,
        'JAS' : 8,
        'ASO' : 9,
        'SON' : 10,
        'OND' : 11,
        'NDJ' : 12
    }
    return oni_mon_map[example]

In [6]:
oni_df = pd.read_fwf('https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt', sep=' ')
# oni_df = oni_df[['Unnamed: 2', 'YR', 'Unnamed: 5', 'Unnamed: 7']]
oni_df.columns = ['oni_mon', 'year', 'sst34', 'oni']
oni_df['oni_mon'] = oni_df['oni_mon'].map(lambda _ : oni_mon_to_number(_))

In [8]:
oni_df = oni_df[['oni_mon', 'year', 'oni']]
oni_df = oni_df.rename(columns={'oni_mon' : 'month'})

DMI/IOD Update

In [10]:
def dmi_mon_to_number(example) : 
    dmi_mon_map = {
        'jan' : 1,
        'feb' : 2,
        'mar' : 3,
        'apr' : 4,
        'may' : 5,
        'jun' : 6,
        'jul' : 7,
        'aug' : 8,
        'sep' : 9,
        'oct' : 10,
        'nov' : 11,
        'dec' : 12
    }
    return dmi_mon_map[example]

In [11]:
dmi_df = pd.read_csv('http://tiservice.hii.or.th/opendata/clmidx/dmi_had.csv', na_values=['-9999.000'])
# dmi_df = pd.read_csv('/content/drive/Shareddrives/Suakaw/workspace/HII/SimIDX/downloaded/dmi_had.csv', na_values=['-9999.000'])
dmi_df = dmi_df.melt(id_vars=['year'], value_vars=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])
dmi_df = dmi_df.rename(columns={'variable': 'month', 'value' : 'iod'})
dmi_df['month'] = dmi_df['month'].map(lambda _: dmi_mon_to_number(_))
dmi_df = dmi_df.sort_values(['year','month'])
dmi_df = dmi_df[['month', 'year', 'iod']]

In [12]:
dmi_df

,month,year,iod
0,1,1870,-0.373
153,2,1870,-0.256
306,3,1870,0.277
459,4,1870,0.027
612,5,1870,-0.400
...,...,...,...
1223,8,2022,NaN
1376,9,2022,NaN
1529,10,2022,NaN
1682,11,2022,NaN


PDO Update

In [13]:
pdo_df = pd.read_table('https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat', skiprows=1, sep="\s+")

In [14]:
for co in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'] :
    pdo_df[co] = pdo_df[co].apply(lambda _ : float(str(_).replace('-99.99', '')))

In [15]:
pdo_df.columns = ['year', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
pdo_df = pdo_df.melt(id_vars=['year'], value_vars=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])
pdo_df = pdo_df.rename(columns={'variable': 'month', 'value' : 'pdo'})
pdo_df['month'] = pdo_df['month'].map(lambda _: dmi_mon_to_number(_))
pdo_df = pdo_df.sort_values(['year','month'])
pdo_df = pdo_df[['month', 'year', 'pdo']]

In [16]:
pdo_df

,month,year,pdo
0,1,1854,0.11
169,2,1854,-0.24
338,3,1854,-0.40
507,4,1854,-0.44
676,5,1854,-0.54
...,...,...,...
1351,8,2022,NaN
1520,9,2022,NaN
1689,10,2022,NaN
1858,11,2022,NaN


In [22]:
import requests
import re
import io

In [32]:
emifile_data = requests.get('https://apcc21.org/cmm/fms/FileDown2.do;jsessionid=5FA508AC0BA3B7435DCB6BCAB7C5F527?atchFileId=EMI_2D.txt&fileSn=1').text

In [33]:
emifile_data = [ _ for _ in emifile_data.split('\n')]

In [36]:
emifile_data = [ re.sub(r'\s+', ',', _) for _ in emifile_data ]

emifile_data_str = functools.reduce(lambda a,b: a+'\n'+b, emifile_data)

with io.StringIO() as f :
    f.write(emifile_data_str)
    f.seek(0)
    emi_df = pd.read_csv(f, na_values=-999.999)

In [37]:
emi_df

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1948,-0.089,-0.256,-1.976,0.335,0.596,0.580,0.112,0.026,0.150,0.229,0.224,0.397
1,1949,0.251,-0.016,-0.024,-0.099,0.141,0.034,0.303,0.203,0.252,-0.064,-0.212,-0.332
2,1950,-0.222,-0.322,-0.777,-0.733,-0.384,-0.081,-0.283,-0.382,-0.356,-0.398,-0.356,-0.277
3,1951,-0.390,-0.357,0.122,0.002,0.494,0.061,0.087,-0.200,-0.182,-0.336,-0.101,0.074
4,1952,0.058,-0.100,-0.060,0.267,0.383,0.263,0.046,0.032,0.370,-0.125,0.114,-0.037
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2018,-0.187,-0.087,-0.020,0.097,0.132,0.303,0.279,0.418,0.479,0.541,0.372,0.284
71,2019,0.357,0.430,0.415,0.304,0.234,0.447,0.729,0.777,0.744,0.730,0.611,0.588
72,2020,0.529,0.359,0.437,0.223,0.022,0.182,0.102,-0.228,-0.206,-0.376,-0.442,-0.640
73,2021,-0.775,-0.657,-0.459,-0.158,0.014,-0.042,-0.242,-0.376,-0.436,-0.572,-0.465,-0.165


In [38]:
emi_df.columns = ['year'] + ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
emi_df = emi_df.melt(id_vars=['year'], value_vars=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])
emi_df = emi_df.rename(columns={'variable': 'month', 'value' : 'emi'})
emi_df['month'] = emi_df['month'].map(lambda _: dmi_mon_to_number(_))
emi_df = emi_df.sort_values(['year','month'])
emi_df = emi_df[['month', 'year', 'emi']]

In [39]:
emi_df

,month,year,emi
0,1,1948,-0.089
75,2,1948,-0.256
150,3,1948,-1.976
225,4,1948,0.335
300,5,1948,0.596
...,...,...,...
599,8,2022,NaN
674,9,2022,NaN
749,10,2022,NaN
824,11,2022,NaN
